In [3]:
import zapisywacz as zap
#import model_dnn as mod
import numpy as np
import matplotlib.pyplot as plt
import read_tree as rt

Welcome to JupyROOT 6.15/01


In [4]:
sciezka_sygnalu="data_rootowe_obrobione/MT_GluGlu.root"
sciezka_tla="data_rootowe_obrobione/MT_WJets.root"

In [5]:
def wczytaj_do_pythonowych(sciezka_rootowego):
    """funkcja zwraca pythonowe w wygodnej do uzycia formie"""
    legs, jets, global_params, properties = rt.read_tree(file_name=sciezka_rootowego)
    #te rzeczy sa zle, bo maja niewygodny format oraz global_params["nJets30"] ma
    #zdublowane dane, to znaczy [1,1,3,3,5,5 ...]
    #mozna to sprawdzic ta metoda
    #nJets30 ma od 0 do 6 a nawet do 8,
    #leg_1_charge ma -1 oraz 1
    #leg_2_charge ma -1 oraz 1
    #'leg_2_decayMode' ma 0,1,10
    kategoryczne=["nJets30","leg_1_charge","leg_2_charge",'leg_2_decayMode']
    zdublowany_klucz="nJets30"
    def czy_zdublowane(tab):
        for i in range(len(tab)//2):
            if tab[2*i] != tab[2*i+1]:
                print ("kontrprzylad dla tezy, ze to jest zdublowana lista")
                assert False
    czy_zdublowane(global_params[zdublowany_klucz])
    def wyintuj_kategoryczne(slownik):
        for k in slownik.keys():
            if k in kategoryczne:
                slownik[k]=np.array(slownik[k],dtype=int)
    wyintuj_kategoryczne(global_params)
    wyintuj_kategoryczne(properties)
    odrzucane_rzeczy=["BJetBetaScore"]
    #jets 1 jest zle, wszystkie dane tam sa stale chyba
    del jets[1]
    def wyrzuc_odrzucone(slownik):
        klucze=list(slownik.keys())
        for k in klucze:
            if k in odrzucane_rzeczy:
                del slownik[k]
    wyrzuc_odrzucone(global_params)
    wyrzuc_odrzucone(properties)
    do_zlogarytmowania_z_dodaniem_0_01=["higgsMassTrans","higgsPT","visMass",
                                        "leg_1_combreliso","leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits"]
    def zlogarytmoj(slownik):
        for k in slownik.keys():
            if k in do_zlogarytmowania_z_dodaniem_0_01:
                slownik[k]=np.log(np.array(slownik[k])+0.01)
    zlogarytmoj(global_params)
    zlogarytmoj(properties)
    def zlogarytmoj_energie(tab):
        for i in range(len(tab)):
            tab[i][0]=np.log(np.array(tab[i][0])+0.01)
    zlogarytmoj_energie(legs)
    zlogarytmoj_energie(jets)
    def f1(x):
        return np.log((1.-x))
    do_zrobienia_f1=["leg_2_byIsolationMVArun2v1DBoldDMwLTraw"]
    def zrob_f1(slownik):
        for k in slownik.keys():
            if k in do_zrobienia_f1:
                slownik[k]=f1(np.array(slownik[k]))
    zrob_f1(global_params)
    zrob_f1(properties)
    def przerob_legs_lub_jets(tab):
        tab=np.array(tab)
        return np.transpose(tab,(2,0,1))
    def przerob_global_params_lub_properties(slownik):
        wyrzut=[]
        jeden_z_kluczy=list(slownik.keys())[0]
        #uwaga hakerstwo
        liczba_przypadkow=0
        if jeden_z_kluczy==zdublowany_klucz:
            liczba_przypadkow=len(slownik[jeden_z_kluczy]) //2
        else:
            liczba_przypadkow=len(slownik[jeden_z_kluczy])
        for i in range(liczba_przypadkow):
            doczepiany={}
            for k in slownik.keys():
                if k != zdublowany_klucz:
                    doczepiany[k]=slownik[k][i]
                else:
                    doczepiany[k]=slownik[k][2*i]
            wyrzut.append(doczepiany)
        return wyrzut
    legs=przerob_legs_lub_jets(legs)
    jets=przerob_legs_lub_jets(jets)
    global_params=przerob_global_params_lub_properties(global_params)
    properties=przerob_global_params_lub_properties(properties)
    return legs, jets, global_params, properties

In [19]:
slownik_kategorycznych={"nJets30":[0,1,2,3,4,5,6,7,8],
                       "leg_1_charge":[-1,1],
                       "leg_2_charge":[-1,1],
                       "leg_2_decayMode":[0,1,10]}

In [6]:
legs_syg, jets_syg, global_params_syg, properties_syg=\
wczytaj_do_pythonowych(sciezka_sygnalu)
legs_tlo, jets_tlo, global_params_tlo, properties_tlo=\
wczytaj_do_pythonowych(sciezka_tla)

[ML]	Opening root file for python conversion.
[ML]	Reading data from TTree.
[ML]	Conversion to python successful!
[ML]	Opening root file for python conversion.
[ML]	Reading data from TTree.
[ML]	Conversion to python successful!


In [10]:
liczba_przypadkow_tlo=len(legs_tlo)
liczba_przypadkow_syg=len(legs_syg)
print(liczba_przypadkow_tlo)
print(liczba_przypadkow_syg)

3087
59083


In [14]:
print (legs_syg.shape)
print (jets_syg.shape)

(59083, 2, 4)
(59083, 1, 4)


In [17]:
global_params_syg[0]

{'higgsMassTrans': 2.6793719558045055,
 'higgsPT': 2.901246907610941,
 'nJets30': 0,
 'visMass': 3.9368709061941067}

In [18]:
properties_syg[0]

{'leg_1_charge': 1,
 'leg_1_combreliso': -3.3837063308792876,
 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': -4.605170185988091,
 'leg_2_byIsolationMVArun2v1DBoldDMwLTraw': -3.542529797871357,
 'leg_2_charge': -1,
 'leg_2_decayMode': 1}

In [28]:
def rob_numeryczne(legs, jets, global_params, properties):
    w = {}
    w['legs'] = len(legs[0].reshape((-1,)))
    w['jets'] = len(jets[0].reshape((-1,)))
    for s in [global_params, properties]:
        for k in s[0].keys():
            if (not (k in slownik_kategorycznych)):
                w[k] = 1
    return w

In [29]:
slownik_numerycznych = rob_numeryczne(legs_syg, jets_syg, global_params_syg
                                ,properties_syg)

In [30]:
slownik_numerycznych 

{'legs': 8,
 'jets': 4,
 'higgsMassTrans': 1,
 'higgsPT': 1,
 'visMass': 1,
 'leg_1_combreliso': 1,
 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': 1,
 'leg_2_byIsolationMVArun2v1DBoldDMwLTraw': 1}

In [32]:
def rob_jeden_przypadek(i, legs, jets, global_params, properties):
    n = {}
    c = {}
    n['legs'] = legs[i].reshape((-1,))
    n['jets'] = jets[i].reshape((-1,))
    for s in [global_params, properties]:
        for k in s[i].keys():
            if ( k in slownik_kategorycznych):
                c[k] = s[i][k]
            else:
                n[k] = s[i][k]
    return n, c

In [33]:
rob_jeden_przypadek(5, legs_syg, jets_syg, global_params_syg, 
                   properties_syg)

({'legs': array([  3.39761832,  16.25114959,  14.21720409,  20.65775908,
           4.45590599, -32.5085886 , -45.41972008, -65.55087095]),
  'jets': array([-4.60517019,  0.        ,  0.        ,  0.        ]),
  'higgsMassTrans': 3.620412168348662,
  'higgsPT': 3.9811968611061324,
  'visMass': 4.6153824291135415,
  'leg_1_combreliso': -2.3230754541619603,
  'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': -0.44945209087010646,
  'leg_2_byIsolationMVArun2v1DBoldDMwLTraw': -2.931676502631464},
 {'nJets30': 1, 'leg_1_charge': 1, 'leg_2_charge': -1, 'leg_2_decayMode': 1})

In [39]:
ln_syg, lk_syg, wr_syg = zap.tworz_folder_do_zapisu('sygnal', slownik_numerycznych, 
                          slownik_kategorycznych)
ln_tlo, lk_tlo, wr_tlo = zap.tworz_folder_do_zapisu('tlo', slownik_numerycznych,
                          slownik_kategorycznych)

In [40]:
for i in range(liczba_przypadkow_syg):
    n,c = rob_jeden_przypadek(i, legs_syg, jets_syg, 
                             global_params_syg, 
                             properties_syg)
    zap.zapisz_jeden_przyklad(n, c, 1, ln_syg, lk_syg, wr_syg)
wr_syg.close()

In [41]:
for i in range(liczba_przypadkow_tlo):
    n,c = rob_jeden_przypadek(i, legs_tlo, jets_tlo, 
                             global_params_tlo, 
                             properties_tlo)
    zap.zapisz_jeden_przyklad(n, c, 0, ln_tlo, lk_tlo, wr_tlo)
wr_tlo.close()


In [42]:
#teraz mieszanie tych danych

In [43]:
import tensorflow as tf

In [44]:
import model_dnn as mod

In [50]:

def rob_pary_indexow_syg_tlo(od_kad, do_kad):
    od_kad_syg = int(od_kad * liczba_przypadkow_syg)
    od_kad_tlo = int(od_kad * liczba_przypadkow_tlo)
    
    do_kad_syg = int(do_kad * liczba_przypadkow_syg)
    do_kad_tlo = int(do_kad * liczba_przypadkow_tlo)
    
    lista = []
    for i in range(do_kad_syg - od_kad_syg):
        lista.append((
            od_kad_syg + i,
            od_kad_tlo + (i % (do_kad_tlo - od_kad_tlo))))
    return lista
    

In [51]:
ln_tr, lk_tr, wr_tr = zap.tworz_folder_do_zapisu('train', slownik_numerycznych, 
                          slownik_kategorycznych)
ln_va, lk_va, wr_va = zap.tworz_folder_do_zapisu('val', slownik_numerycznych, 
                          slownik_kategorycznych)
ln_te, lk_te, wr_te = zap.tworz_folder_do_zapisu('test', slownik_numerycznych, 
                          slownik_kategorycznych)

In [54]:
def zapisz(od_kad, do_kad, ln, lk, wr):
    lista = rob_pary_indexow_syg_tlo(od_kad, do_kad)
    for para in lista:
        in_syg, in_tlo = para
        n,c = rob_jeden_przypadek(in_syg, legs_syg, jets_syg, 
                             global_params_syg, 
                             properties_syg)
        
        zap.zapisz_jeden_przyklad(n, c, 1, ln, lk, wr)
        n,c = rob_jeden_przypadek(in_tlo, legs_tlo, jets_tlo, 
                             global_params_tlo, 
                             properties_tlo)
        
        zap.zapisz_jeden_przyklad(n, c, 0, ln, lk, wr)
    
        

In [55]:
zapisz(0., 0.8, ln_tr, lk_tr, wr_tr)

In [57]:
zapisz(0.8, 0.9, ln_va, lk_va, wr_va)

In [58]:
zapisz(0.9, 1., ln_te, lk_te, wr_te)

In [59]:
wr_tr.close()
wr_te.close()
wr_va.close()